In [8]:
import ast  # Módulo de Árboles de Sintaxis Abstractos
import json  # Módulo json para codificación y decodificación
import pandas as pd  # Módulo pandas para Análisis de Datos
import nltk  # Módulo nltk - Natural Language Toolkit


# Desanidamos los archivos

## Archivo australian_user_reviews

In [9]:
# Lista para almacenar los diccionarios json de cada línea
reviews = []



# Ruta del archivo json
with open('D:\\Curso Henry\\PI\\australian_user_reviews.json', 'r', encoding='utf-8') as f:
    # bucle para insertar datos en la lista reviews
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))

# creación del DataFrame a partir de la lista
df_reviews = pd.DataFrame(reviews)

# visualización de los primeros registros
df_reviews



,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [10]:
# revisión de valores nulos
df_reviews.isna().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

In [11]:
#Eliminar las columnas que no vamos a utilizar para nuestro modelo
df_reviews = df_reviews.drop(['user_url'], axis=1)
df_reviews

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...
25794,76561198306599751,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Las siguientes dos líneas de código se utilizan para transformar un DataFrame que contiene una columna de listas en un DataFrame que contiene una fila para cada elemento de la lista.

In [12]:
# desanidar el diccionario de la columna reviews
desanidando = df_reviews.explode('reviews') # cada elemento de la lista en la columna 'reviews' se convierte en una fila separada, y las demás columnas se replican en cada fila resultante.
df_reviews_user = pd.concat([desanidando.drop(['reviews'], axis=1), desanidando['reviews'].apply(pd.Series)], axis=1) # combinar las columnas del DataFrame desanidando con las columnas de una nueva tabla que se crea a partir de la columna 'reviews' del DataFrame desanidando.

In [13]:
desanidando

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...
25797,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [14]:
df_reviews_user

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [15]:

df_reviews_user = df_reviews_user.drop(df_reviews_user.columns[-1], axis=1) # eliminar la última columna del DataFrame df_reviews_user
df_reviews_user.drop(['funny', 'last_edited'], axis=1, inplace=True) # eliminar las columnas 'funny' y 'last_edited' del DataFrame df_reviews_user

In [16]:
df_reviews_user

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
25797,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


In [17]:

df_reviews_user['year_posted'] = df_reviews_user['posted'].str.extract(r'(\d{4})') # extraer el año de la columna 'posted' y almacenarlo en una nueva columna 'year_posted'

df_reviews_user.drop('posted' , axis = 1, inplace = True) # eliminar la columna 'posted' del DataFrame df_reviews_user

In [18]:
df_reviews_user

,user_id,item_id,helpful,recommend,review,year_posted
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011
0,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
1,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
25797,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,730,No ratings yet,True,:D,NaN


In [19]:

nltk.download('vader_lexicon') # descarga del diccionario vader_lexicon 
from nltk.sentiment.vader import SentimentIntensityAnalyzer # importar el analizador de sentimientos vader

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [20]:
analizador_sentimientos = SentimentIntensityAnalyzer() # inicializar el analizador de sentimientos vader

def puntaje_sentimiento(texto): 
    if pd.isnull(texto) or texto == '':
        return 1  # Retorna 1 si el texto es nulo o vacío
    elif isinstance(texto, str):
        sentimiento = analizador_sentimientos.polarity_scores(texto) 
        puntaje_compuesto = sentimiento['compound'] 
        if puntaje_compuesto >= -0.05: # si el puntaje compuesto es mayor o igual a -0.05, se considera un buen puntaje
            return 2  
        elif puntaje_compuesto <= -0.05: # si el puntaje compuesto es menor o igual a -0.05, se considera un mal puntaje
            return 0 
        else:
            return 1
    else:
        return 1 # Retorna 1 si el texto no es un string

In [21]:
# convertir la columna 'review' en string
df_reviews_user['review'] = df_reviews_user['review'].astype(str) 

# aplicar la función puntaje_sentimiento a la columna 'review' y almacenar el resultado en la columna 'sentiment_score'.
df_reviews_user['sentiment_score'] = df_reviews_user['review'].apply(puntaje_sentimiento) 


df_reviews_user


,user_id,item_id,helpful,recommend,review,year_posted,sentiment_score
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,2
0,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2
0,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,2
1,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,2
...,...,...,...,...,...,...,...
25797,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN,2
25797,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN,2
25798,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN,2
25798,LydiaMorley,730,No ratings yet,True,:D,NaN,2


In [22]:
# revisamos los valores nulos
df_reviews_user.isnull().sum()

user_id                0
item_id               28
helpful               28
recommend             28
review                 0
year_posted        10147
sentiment_score        0
dtype: int64

In [23]:
# Para las columnas 'item_id', 'helpful' y 'recommend' eliminamos los registros que tengan valores nulos
df_reviews_user.dropna(subset=['item_id', 'helpful', 'recommend'], inplace=True)

Imputamos valores para la columna year_posted, pues son muchos datos para eliminarlos (22%).

Dado que la distribución de los valores en la columna posted_year no es simétrica, la mediana es la mejor opción para imputar los valores nulos. La mediana es menos sensible a los valores atípicos y puede proporcionar una mejor representación de la distribución de los datos.

In [24]:
# el porcentaje de cada uno de los valores de la columna 'year_posted'
df_reviews_user['year_posted'].value_counts(normalize=True)

year_posted
2014    0.449538
2015    0.375513
2013    0.138088
2012    0.024702
2011    0.010816
2010    0.001342
Name: proportion, dtype: float64

In [25]:
# Imputamos los valores nulos de la columna 'year_posted' con el valor 2013 que es el valor de la mediana
df_reviews_user['year_posted'].fillna(2013, inplace=True)

In [26]:
# volvemos a revisar los valores nulos
df_reviews_user.isnull().sum()

user_id            0
item_id            0
helpful            0
recommend          0
review             0
year_posted        0
sentiment_score    0
dtype: int64

In [27]:
# Guardar el DataFrame df_reviews_user en un archivo CSV llamado 'df_reviews_user.csv'
# Omitir la columna de índices al guardar
df_reviews_user.to_csv('df_reviews_user.csv', index=False)


In [28]:
df_reviews_user

,user_id,item_id,helpful,recommend,review,year_posted,sentiment_score
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,2
0,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2
0,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,2
1,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,2
...,...,...,...,...,...,...,...
25797,76561198312638244,70,No ratings yet,True,a must have classic from steam definitely wort...,2013,2
25797,76561198312638244,362890,No ratings yet,True,this game is a perfect remake of the original ...,2013,2
25798,LydiaMorley,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,2013,2
25798,LydiaMorley,730,No ratings yet,True,:D,2013,2


In [ ]:
# Suponiendo que df_reviews_user es tu DataFrame
df_reviews_user = df_reviews_user.drop('helpful', axis=1)


In [31]:

df_reviews_user.to_csv('df_reviews_user.csv', index=False)
